Download pretrained model embedding Glove


In [ ]:
!pip install --upgrade gensim

: 

In [ ]:
!wget http://nlp.stanford.edu/data/glove.6B.zip

--2024-04-11 03:59:19--  http://nlp.stanford.edu/data/glove.6B.zip
Resolving nlp.stanford.edu (nlp.stanford.edu)... 171.64.67.140
Connecting to nlp.stanford.edu (nlp.stanford.edu)|171.64.67.140|:80... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://nlp.stanford.edu/data/glove.6B.zip [following]
--2024-04-11 03:59:19--  https://nlp.stanford.edu/data/glove.6B.zip
Connecting to nlp.stanford.edu (nlp.stanford.edu)|171.64.67.140|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: https://downloads.cs.stanford.edu/nlp/data/glove.6B.zip [following]
--2024-04-11 03:59:20--  https://downloads.cs.stanford.edu/nlp/data/glove.6B.zip
Resolving downloads.cs.stanford.edu (downloads.cs.stanford.edu)... 171.64.64.22
Connecting to downloads.cs.stanford.edu (downloads.cs.stanford.edu)|171.64.64.22|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 862182613 (822M) [application/zip]
Saving to: ‘glove.6B.zip’

glov

In [ ]:
!unzip glove*.zip

Archive:  glove.6B.zip
  inflating: glove.6B.50d.txt        
  inflating: glove.6B.100d.txt       
  inflating: glove.6B.200d.txt       
  inflating: glove.6B.300d.txt       


In [ ]:
%ls

data/              glove.6B.200d.txt  glove.6B.50d.txt  sample_data/
glove.6B.100d.txt  glove.6B.300d.txt  glove.6B.zip


In [ ]:
from gensim.scripts.glove2word2vec import glove2word2vec
glove_input_file = 'glove.6B.100d.txt'
word2vec_output_file = 'word2vec.txt'
glove2word2vec(glove_input_file, word2vec_output_file)

<ipython-input-6-85b6fe2dabac>:4: DeprecationWarning: Call to deprecated `glove2word2vec` (KeyedVectors.load_word2vec_format(.., binary=False, no_header=True) loads GLoVE text vectors.).
  glove2word2vec(glove_input_file, word2vec_output_file)


(400000, 100)

In [ ]:
from torchtext import vocab
word2vec = vocab.Vectors('glove.6B.100d.txt', 'train/')

100%|█████████▉| 399999/400000 [00:27<00:00, 14612.97it/s]


In [ ]:
print(word2vec['king'].shape)
word2vec['<pad>']

torch.Size([100])


tensor([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0.])

In [ ]:
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [ ]:
import pandas as pd
import torch
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data.dataset import Dataset
from torch.utils.data import DataLoader
from nltk.tokenize import sent_tokenize, word_tokenize
import csv
import numpy as np

In [ ]:
class CustomDataset(Dataset):

    def __init__(self, data_dir, max_length_word=50, max_length_sentences=16, transform=None, target_transform=None):
        super(CustomDataset, self).__init__()

        self.data_dir = data_dir
        self.max_length_sentences = max_length_sentences
        self.max_length_word = max_length_word
        self.transform = transform
        self.target_transform = target_transform

        text_list, label_list = [], []
        with open(data_dir) as csv_file:
            reader = csv.reader(csv_file, quotechar='"')
            for idx, line in enumerate(reader):
                if idx == 0:
                    continue
                text = ""
                for tx in line[1:]:
                    text += tx.lower()
                    text += ". "
                label = int(line[0])-1
                text_list.append(text)
                label_list.append(label)

        self.label_list = label_list
        self.text_list = text_list

    def __len__(self):
        return len(self.label_list)

    def __getitem__(self, index):
        label = self.label_list[index]
        text = self.text_list[index]
        document_encode = [[word2vec[word] for word in word_tokenize(sentences)]
                           for sentences in sent_tokenize(text)]

        for sentences in document_encode:
            if len(sentences) < self.max_length_word:
                extended_words = [word2vec['<pad>'].numpy() for _ in range(self.max_length_word - len(sentences))]
                sentences.extend(extended_words)

        if len(document_encode) < self.max_length_sentences:
            extended_sentences = [[word2vec['<pad>'].numpy() for _ in range(self.max_length_word)] for _ in
                                  range(self.max_length_sentences - len(document_encode))]
            document_encode.extend(extended_sentences)

        document_encode = [sentences[:self.max_length_word] for sentences in document_encode][
                          :self.max_length_sentences]

        document_encode = np.stack(arrays=document_encode, axis=0)
        if self.transform:
            document_encode = self.transform(document_encode)
        if self.target_transform:
            label = self.target_transform(label)

        return document_encode, label

In [ ]:
texts, labels = [], []
data_path="/content/data/test.csv"
with open(data_path) as csv_file:
    reader = csv.reader(csv_file, quotechar='"')
    for idx, line in enumerate(reader):
        if idx == 0:
            continue
        text = ""
        for tx in line[1:]:
            text += tx.lower()
            text += ". "
        label = int(line[0])
        texts.append(text)
        labels.append(label)

n = len(labels)
print (n)

7600


In [ ]:
from torchtext.functional import to_tensor
from torchvision.transforms import ToTensor, Lambda

training_data = CustomDataset(data_dir="/content/data/train.csv",
                              transform = torch.tensor,
                              target_transform=Lambda(lambda y: torch.zeros(4, dtype=torch.float).scatter_(0, torch.tensor(y), value=1))
)
test_data = CustomDataset(data_dir="/content/data/test.csv",
                          transform = torch.tensor,
                          target_transform=Lambda(lambda y: torch.zeros(4, dtype=torch.float).scatter_(0, torch.tensor(y), value=1))
)

train_dataloader = DataLoader(training_data, batch_size=128, shuffle=True)
test_dataloader = DataLoader(test_data, batch_size=128, shuffle=True)

print(len(train_dataloader))
for batch in test_dataloader:
    print((batch[1]).shape)
    break

938
torch.Size([128, 4])


In [ ]:
class SentAttNet(nn.Module):
    def __init__(self,  hidden_size=50):
        super(SentAttNet, self).__init__()
        self.sent_weight = nn.Parameter(torch.Tensor(2 * hidden_size, 10))
        self.sent_bias = nn.Parameter(torch.Tensor(10))
        self.context_weight = nn.Parameter(torch.Tensor(10, 1))
        self.gru = nn.GRU(100, hidden_size, bidirectional=True)
        self._create_weights(mean=0.0, std=0.05)

    def _create_weights(self, mean=0.0, std=0.05):
        self.sent_weight.data.normal_(mean, std)
        self.context_weight.data.normal_(mean, std)

    def forward(self, input, hidden_state):
        f_output, h_output = self.gru(input, hidden_state)
        output = matrix_mul(f_output, self.sent_weight, self.context_weight, self.sent_bias)
        output = F.softmax(output)
        output = element_wise_mul(output.permute(1,0,2), f_output.permute(1,0,2))

        return output, h_output

In [ ]:
class WordAttNet(nn.Module):
    def __init__(self,  hidden_size=50):
        super(WordAttNet, self).__init__()
        self.word_weight = nn.Parameter(torch.Tensor(2 * hidden_size, 10))
        self.word_bias = nn.Parameter(torch.Tensor(10))
        self.context_weight = nn.Parameter(torch.Tensor(10, 1))
        self.gru = nn.GRU(100, hidden_size, bidirectional=True)
        self._create_weights(mean=0.0, std=0.05)

    def _create_weights(self, mean=0.0, std=0.05):
        self.word_weight.data.normal_(mean, std)
        self.context_weight.data.normal_(mean, std)

    def forward(self, input, hidden_state):
        f_output, h_output = self.gru(input, hidden_state)
        output = matrix_mul(f_output, self.word_weight, self.context_weight, self.word_bias)
        output = F.softmax(output)
        output = element_wise_mul(output.permute(1,0,2), f_output.permute(1,0,2))
        return output, h_output

def element_wise_mul(input1, input2):
    feature_list = []
    for feature_1, feature_2 in zip(input1, input2):
        feature = (feature_1 * feature_2).sum(dim=0)
        feature_list.append(feature)

    output = torch.stack(feature_list, dim=0)
    return output

def matrix_mul(input, weight, context_weight,  bias=False):
    input = torch.matmul(input, weight)
    input = input + bias
    input = torch.tanh(input)
    input = torch.matmul(input, context_weight)

    return input

In [ ]:
class HierAttNet(nn.Module):
    def __init__(self, word_hidden_size=50, sent_hidden_size=50, batch_size=128, num_classes=4, max_sent_length=16, max_word_length=50):
        super(HierAttNet, self).__init__()
        self.batch_size = batch_size
        self.word_hidden_size = word_hidden_size
        self.sent_hidden_size = sent_hidden_size
        self.max_sent_length = max_sent_length
        self.max_word_length = max_word_length
        self.word_att_net = WordAttNet(word_hidden_size)
        self.sent_att_net = SentAttNet(sent_hidden_size)
        self._init_hidden_state()
        self.dense = torch.nn.Linear(sent_hidden_size*2, num_classes, bias=True)

    def _init_hidden_state(self, last_batch_size=None):
        if last_batch_size:
            batch_size = last_batch_size
        else:
            batch_size = self.batch_size
        self.word_hidden_state = torch.zeros(2, batch_size, self.word_hidden_size)
        self.sent_hidden_state = torch.zeros(2, batch_size, self.sent_hidden_size)
        if torch.cuda.is_available():
            self.word_hidden_state = self.word_hidden_state.cuda()
            self.sent_hidden_state = self.sent_hidden_state.cuda()

    def forward(self, input):
        output_list = []
        input = input.permute(1, 0, 2, 3)
        for seq in input:
            output, self.word_hidden_state = self.word_att_net(seq.permute(1, 0, 2), self.word_hidden_state)
            output_list.append(output)

        output = torch.stack(output_list, dim=0)
        output, self.sent_hidden_state = self.sent_att_net(output, self.sent_hidden_state)
        output = self.dense(output)
        output = F.softmax(output,dim=1)

        return output

In [ ]:
model = HierAttNet()
print(model)

HierAttNet(
  (word_att_net): WordAttNet(
    (gru): GRU(100, 50, bidirectional=True)
  )
  (sent_att_net): SentAttNet(
    (gru): GRU(100, 50, bidirectional=True)
  )
  (dense): Linear(in_features=100, out_features=4, bias=True)
)


# Train

In [ ]:
import time
from tqdm import tqdm

def train_loop(model, dataloader, optimizer, loss_fn, epoch_num):
    size = len(dataloader)
    total_loss = 0
    correct = 0
    model.train()
    model.zero_grad()

    for batch in tqdm(dataloader, f"Epoch: {epoch_num}"):
        batch = [t.to(device='cuda') for t in batch]
        if batch[1].shape[0] < 128:
              model._init_hidden_state(last_batch_size = batch[1].shape[0])
        else:
              model._init_hidden_state()
        optimizer.zero_grad()

        pred = model(batch[0])
        loss = loss_fn(pred, batch[1])
        loss.backward()
        optimizer.step()

        pred_classes = torch.zeros_like(pred)
        pred_classes[torch.arange(len(pred)), torch.argmax(pred, dim=1)] = 1
        correct_preds = (pred_classes * batch[1]).float()

        correct += (correct_preds.sum()/len(correct_preds)).item()
        total_loss += loss.item()

        time.sleep(0.001) # for tqdm progess bar

    print(f"\nTrain Error:\t Accuracy: {(100*correct/size):>0.1f}%, Avg loss: {total_loss/size:>8f} \n")


def test_loop(model, dataloader, loss_fn, epoch_num):
    size = len(dataloader)
    total_loss = 0
    correct = 0
    model.eval()

    with torch.no_grad():
        for batch in dataloader:
            batch = [t.to(device='cuda') for t in batch]
            if batch[1].shape[0] < 128:
                model._init_hidden_state(last_batch_size = batch[1].shape[0])
            else:
                model._init_hidden_state()

            pred = model(batch[0])
            loss = loss_fn(pred, batch[1])

            pred_classes = torch.zeros_like(pred)
            pred_classes[torch.arange(len(pred)), torch.argmax(pred, dim=1)] = 1
            correct_preds = (pred_classes * batch[1]).float()

            correct += (correct_preds.sum()/len(correct_preds)).item()
            total_loss += loss.item()

    print(f"\tTest Error:\t Accuracy: {(100*correct/size):>0.1f}%, Avg loss: {total_loss/size:>8f} \n")

In [ ]:
model = HierAttNet()
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
lr = 3e-4
optimizer = torch.optim.Adam(model.parameters(),lr=lr)
loss_fn = torch.nn.CrossEntropyLoss()
model.to(device)

epochs = 3
for epoch in range(epochs):
    print(f'EPOCH {epoch+1}:')
    torch.compile(train_loop(model, train_dataloader, optimizer, loss_fn, epoch+1))
    test_loop(model, test_dataloader, loss_fn, epoch+1)


torch.save(model.state_dict(), 'model.pth')

EPOCH 1:


Epoch: 1:   0%|          | 0/938 [00:00<?, ?it/s]<ipython-input-87-f0a3bdbdf28b>:17: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  output = F.softmax(output)
Epoch: 1: 100%|██████████| 938/938 [20:17<00:00,  1.30s/it]



Train Error:	 Accuracy: 77.5%, Avg loss: 0.974136 

	Test Error:	 Accuracy: 87.5%, Avg loss: 0.867958 

EPOCH 2:


Epoch: 2: 100%|██████████| 938/938 [20:06<00:00,  1.29s/it]



Train Error:	 Accuracy: 88.3%, Avg loss: 0.859467 

	Test Error:	 Accuracy: 88.5%, Avg loss: 0.856519 

EPOCH 3:


Epoch: 3: 100%|██████████| 938/938 [20:33<00:00,  1.31s/it]



Train Error:	 Accuracy: 89.2%, Avg loss: 0.850084 

	Test Error:	 Accuracy: 88.4%, Avg loss: 0.857591 



In [ ]:
model = HierAttNet()
from prettytable import PrettyTable

def count_parameters(model):
    table = PrettyTable(["Modules", "Parameters"])
    total_params = 0
    for name, parameter in model.named_parameters():
        if not parameter.requires_grad:
            continue
        params = parameter.numel()
        table.add_row([name, params])
        total_params += params
    print(table)
    print(f"Total Trainable Params: {total_params}")
    return total_params

count_parameters(model)


+---------------------------------------+------------+
|                Modules                | Parameters |
+---------------------------------------+------------+
|        word_att_net.word_weight       |    1000    |
|         word_att_net.word_bias        |     10     |
|      word_att_net.context_weight      |     10     |
|     word_att_net.gru.weight_ih_l0     |   15000    |
|     word_att_net.gru.weight_hh_l0     |    7500    |
|      word_att_net.gru.bias_ih_l0      |    150     |
|      word_att_net.gru.bias_hh_l0      |    150     |
| word_att_net.gru.weight_ih_l0_reverse |   15000    |
| word_att_net.gru.weight_hh_l0_reverse |    7500    |
|  word_att_net.gru.bias_ih_l0_reverse  |    150     |
|  word_att_net.gru.bias_hh_l0_reverse  |    150     |
|        sent_att_net.sent_weight       |    1000    |
|         sent_att_net.sent_bias        |     10     |
|      sent_att_net.context_weight      |     10     |
|     sent_att_net.gru.weight_ih_l0     |   15000    |
|     sent

93644